# Itertools

The functions in itertools “operate” on iterators to produce more complex iterators. Consider, for example, the built-in zip() function, which takes any number of iterables as arguments and returns an iterator over tuples of their corresponding elements:

```python3
list(zip([1, 2, 3], [a, b, c])
```

How, exactly, does zip() work?

[1, 2, 3] and ['a', 'b', 'c'], like all lists, are iterable, which means they can return their elements one at a time. Technically, any Python object that implements the .\__iter\__() or .\__getitem\__() methods is iterable.
https://docs.python.org/3/glossary.html#term-iterable

Under the hood, the zip() function works, in essence, by calling iter() on each of its arguments, then advancing each iterator returned by iter() with next() and aggregating the results into tuples. The iterator returned by zip() iterates over these tuples.

The map() built-in function is another “iterator operator” that, in its simplest form, applies a single-parameter function to each element of an iterable one element at a time:

```python3
list(map(len, ['abc', 'cd', 'efghi']))
```

The map() function works by calling iter() on its second argument, advancing this iterator with next() until the iterator is exhausted, and applying the function passed to its first argument to the value returned by next() at each step.

```python3
list(map(sum, zip([1, 2, 3], [4, 5, 6])))
```
his is what is meant by the functions in itertools forming an “iterator algebra.” itertools is best viewed as a collection of building blocks that can be combined to form specialized “data pipelines”

There are two main reasons why such an “iterator algebra” is useful: improved memory efficiency (via lazy evaluation) and faster execuction time. To see this, consider the following problem:

Given a list of values inputs and a positive integer n, write a function that splits inputs into groups of length n. For simplicity, assume that the length of the input list is divisible by n. For example, if inputs = [1, 2, 3, 4, 5, 6] and n = 2, your function should return [(1, 2), (3, 4), (5, 6)].


In [1]:
def naive_grouper(inputs, n):
    num_groups = len(inputs) // n
    return [tuple(inputs[i*n:(i+1)*n]) for i in range(num_groups)]

In [20]:
naive_grouper([1, 2, 3, 4, 5, 6], 4)

[(1, 2, 3, 4)]

In [3]:
def better_grouper(inputs, n):
    iters = [iter(inputs)] * n
    return zip(*iters)

In [19]:
list(better_grouper([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 4))

[(1, 2, 3, 4), (5, 6, 7, 8)]

There’s a lot going on in this little function, so let’s break it down with a concrete example. The expression 
```python 3 [iters(inputs)] * n``` creates a list of n references to the same iterator:
```python3
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
iters = [iter(nums)] * 2
list(id(itr) for itr in iters) # IDs are same.
```
Next, zip(*iters) returns an iterator over pairs of corresponding elements of each iterator in iters. When the first element, 1, is taken from the “first” iterator, the “second” iterator now starts at 2 since it is just a reference to the “first” iterator and has therefore been advanced one step. So, the first tuple produced by zip() is (1, 2).

At this point, “both” iterators in iters start at 3, so when zip() pulls 3 from the “first” iterator, it gets 4 from the “second” to produce the tuple (3, 4). This process continues until zip() finally produces (9, 10) and “both” iterators in iters are exhausted:

The better_grouper() function is better for a couple of reasons. First, without the reference to the len() built-in, better_grouper() can take any iterable as an argument (even infinite iterators). Second, by returning an iterator rather than a list, better_grouper() can process enormous iterables without trouble and uses much less memory.



In [9]:
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
iters = [iter(nums)] * 2
list(id(itr) for itr in iters)

[2256410407648, 2256410407648]

In [11]:
list(better_grouper([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 4))

[(1, 2, 3, 4), (5, 6, 7, 8)]

## The grouper Recipe

The problem with better\_grouper() is that it doesn’t handle situations where the value passed to the second argument isn’t a factor of the length of the iterable in the first argument:

```python3
list(better_grouper([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 4))
[(1, 2, 3, 4), (5, 6, 7, 8)]
```
The elements 9 and 10 are missing from the grouped output. This happens because zip() stops aggregating elements once the shortest iterable passed to it is exhausted. It would make more sense to return a third group containing 9 and 10.

To do this, you can use itertools.zip_longest(). This function accepts any number of iterables as arguments and a fillvalue keyword argument that defaults to None. The easiest way to get a sense of the difference between zip() and zip_longest() is to look at some example output:

In [13]:
import itertools as it
x = [1, 2, 3, 4, 5]
y = ['a', 'b', 'c']
list(zip(x, y))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [14]:
list(it.zip_longest(x, y))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, None), (5, None)]

In [17]:
def grouper(inputs, n, fillvalue=None):
    iters = [iter(inputs)] * n
    return it.zip_longest(*iters, fillvalue=fillvalue)

In [18]:
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list(grouper(nums, 4))

[(1, 2, 3, 4), (5, 6, 7, 8), (9, 10, None, None)]

The grouper() function can be found in the Recipes section of the itertools docs. The recipes are an excellent source of inspiration for ways to use itertools to your advantage.
https://docs.python.org/3.6/library/itertools.html#itertools-recipes

Et tu, Brute Force?

Here’s a common interview-style problem:
>You have three \\$20 dollar bills, five \\$10 dollar bills, two \\$5 dollar bills, and five \\$1 dollar bills. How many ways can you make change for a \\$100 dollar bill?
>

```python3
bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]
```

In [24]:
bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]
makes_100 = []

for n in range(1, len(bills)+1):
    for combination in it.combinations(bills, n):
        if sum(combination) == 100:
            makes_100.append(combination)


In [25]:
set(makes_100)

{(20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 10)}

Here’s a variation on the same problem:
> How many ways are there to make change for a \\$100 bill using any number of \\$50, \\$20, \\$10, \\$5, and \\$1 dollar bills?


In [26]:
bills = [50, 20, 10,5, 1]
makes_100 = []
for n in range(1, 101):
    for combination in it.combinations_with_replacement(bills, n):
        if sum(combination) == 100:
            makes_100.append(combination)

In [27]:
len(makes_100)

343

In [29]:
for p in it.permutations('abc'):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


In [30]:
def odds():
    """Generate odd integers, starting with 1."""
    n = 1
    while True:
        yield n
        n += 2

odds = odds()
[next(odds) for _ in range(5)]

[1, 3, 5, 7, 9]

In [31]:
[next(odds) for _ in range(5,10)]

[11, 13, 15, 17, 19]

In [34]:
# using iter tools
odds1 = it.count(start=1, step=2)


[1, 3, 5, 7, 9, 11, 13]

In [35]:
[next(odds1) for _ in range(5,12)]

[15, 17, 19, 21, 23, 25, 27]

In [37]:
list(zip(it.count(), ['a', 'b', 'c']))

[(0, 'a'), (1, 'b'), (2, 'c')]

In [38]:
def fib():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a+b

In [39]:
[next(fib()) for _ in range(5)]

[0, 0, 0, 0, 0]

The accumulate() function takes two arguments—an iterable inputs and a binary function func (that is, a function with exactly two inputs)—and returns an iterator over accumulated results of applying func to elements of inputs. It is roughly equivalent to the following generator:

In [40]:
def accumulate(inputs, func):
    itr = iter(inputs)
    prev = next(itr)
    for cur in itr:
        yield prev
        prev = func(prev, cur)

In [46]:
for i in (accumulate([1, 2, 3, 4, 5], sum)):
    print(i)

1


TypeError: 'int' object is not iterable

In [47]:
import operator
list(it.accumulate([1, 2, 3, 4, 5], operator.add))

[1, 3, 6, 10, 15]

In [48]:
list(it.accumulate([1, 2, 3, 4, 5], lambda x,y: (x + y)/2))

[1, 1.5, 2.25, 3.125, 4.0625]

In [49]:
ranks = ['A', 'K', 'Q', 'J', '10', '9', '8', '7', '6', '5', '4', '3', '2']
suits = ['H', 'D', 'C', 'S']

You could represent a card as a tuple whose first element is a rank and second element is a suit. A deck of cards would be a collection of such tuples. The deck should act like the real thing, so it makes sense to define a generator that yields cards one at a time and becomes exhausted once all the cards are dealt.

In [50]:
def cards():
    """Return a generator that yields playing cards."""
    for rank in ranks:
        for suit in suits:
            yield rank, suit

In [51]:
# implementing same in itertools
cards = it.product(ranks, suits)

In [52]:
import random

def shuffle(deck):
    """Return iterator over shuffle deck."""
    deck = list(deck)
    random.shuffle(deck)
    return iter(tuple(deck))

cards = shuffle(cards)

In [53]:
def cut(deck, n):
    """Return an iterator over a deck of cards cut an index 'n'"""
    if n < 0:
        raise ValueError('`n` nust be a non-negative intefer')
    
    deck = list(deck)
    return iter(deck[n:] + deck[:n])

cards = cut(cards, 26)

The cut() function is pretty simple, but it suffers from a couple of problems. When you slice a list, you make a copy of the original list and return a new list with the selected elements. With a deck of only 52 cards, this increase in space complexity is trivial, but you could reduce the memory overhead using itertools. To do this, you’ll need three functions: itertools.tee(), itertools.islice(), and itertools.chain().

Let’s take a look at how those functions work.

The tee() function can be used to create any number of independent iterators from a single iterable. It takes two arguments: the first is an iterable inputs, and the second is the number n of independent iterators over inputs to return (by default, n is set to 2). The iterators are returned in a tuple of length n.